## Process Outline
We need a set of manually categorized FERC plants with which to test the sklearn Classifier that we're making. We'll also use them to test the regression analysis that Alana is working on. What does this dataset look like, how do we create it?
* Pull FERC Plants table.
* Using a variety of search methods, to ensure that we get a wide variety of plants, identify sets of records in the FERC Form 1 Plants table that are comparable inter-year records.
* Searching can be done, for example, by matching a pattern against the plant name (which ought to show some cases in which the same plant is reported by different respondents, or where the ownership changes from year to year...), or by selecting all of the records from a given respondent, and identifying the records that pertain to a given plant consistently across years, even if the name changes.
* Diversity of plants is key -- we need to show a wide variety of different kinds of time series to the classifier so it knows how to deal with them.

In [334]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [335]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sa
sys.path.append(os.path.abspath(os.path.join('..','..','..')))
from pudl import init, mcoe, analysis, settings, outputs
import pudl.extract.ferc1
import pudl.transform.ferc1
import pudl.constants as pc

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [336]:
# For some reason these things don't stick if they're in the same cell as the
# %matplotlib inline call above, but if they're separate, they work fine.
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 56

In [337]:
ferc1_engine = pudl.extract.ferc1.connect_db()
ferc1_steam_raw = pd.read_sql('SELECT * FROM f1_steam;', ferc1_engine)
fc = pudl.transform.ferc1.FERCPlantClassifier()
fc._prepare_plants(ferc1_steam_raw)
ferc1_prep = fc._ferc1_steam

In [443]:
masks = [
    ((ferc1_prep.plant_name.str.match('.*cayuga.*')) &
     (ferc1_prep.plant_kind_cpi=='steam') &
     (ferc1_prep.yr_const=='1970')),

    ((ferc1_prep.plant_name.str.match('.*comanche.*')) &
     (ferc1_prep.respondent_id==148)),
    
    (ferc1_prep.plant_name.str.match('.*odemach.*2') &
     (ferc1_prep.yr_const=='1982')),
    
    (ferc1_prep.plant_name.str.match('.*klaunion.*') &
    (ferc1_prep.respondent_id==148)),
    
    (ferc1_prep.plant_name.str.match('.*klaunion.*') &
    (ferc1_prep.respondent_id==189)),
    
    (ferc1_prep.plant_name.str.match('.*klaunion.*') &
    (ferc1_prep.respondent_id==24)),
    
    (ferc1_prep.plant_name.str.match('.*effrey.*') &
    (ferc1_prep.respondent_id==191) &
    (ferc1_prep.expns_fuel > 0.0)),
    
    (ferc1_prep.plant_name.str.match('.*effrey.*') &
    (ferc1_prep.respondent_id==80) &
    (ferc1_prep.expns_fuel > 0.0)),
    
    (ferc1_prep.plant_name.str.match('.*effrey.*') &
    (ferc1_prep.respondent_id==182) &
    (ferc1_prep.expns_fuel > 0.0)),
    
    (ferc1_prep.plant_name.str.match('(la cygne 50%|lacygne 50%|lacygne 1.*)') &
    (ferc1_prep.respondent_id==79)),
    
    (ferc1_prep.plant_name.str.match('.*ygne.*2.*') &
    (ferc1_prep.respondent_id==79)),
    
    (ferc1_prep.plant_name.str.match('.*ygne.*#1.*') &
    (ferc1_prep.respondent_id==80)),
    
    (ferc1_prep.plant_name.str.match('.*ygne.*#2.*') &
    (ferc1_prep.respondent_id==80)),
    
    (ferc1_prep.respondent_id==182) &
    (ferc1_prep.plant_name=='sibley'),
    
    ferc1_prep.plant_name.str.match('.*neal.*1'),
    
    ferc1_prep.plant_name.str.match('.*neal.*2'),
    
    ((ferc1_prep.plant_name.str.match('.*neal.*3')) &
    (ferc1_prep.respondent_id==281)),
    
    ((ferc1_prep.plant_name.str.match('.*neal.*3')) &
    (ferc1_prep.respondent_id==210)),
    
    ((ferc1_prep.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
    (ferc1_prep.respondent_id==147)),
    
    ((ferc1_prep.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
    (ferc1_prep.respondent_id==176)),
    
    (ferc1_prep.plant_name.str.match('.*orktown.*')),
    
    (ferc1_prep.plant_name.str.match('chesterfield$')),
    
    ((ferc1_prep.plant_name.str.match('rockport total aeg')) &
                              (ferc1_prep.respondent_id==1)),
    
    ((ferc1_prep.plant_name.str.match('rockport total i&m')) &
                              (ferc1_prep.respondent_id==73)),
    
    ((ferc1_prep.plant_name.str.match('mount storm$')) &
                              (ferc1_prep.respondent_id==186)),
    
    ((ferc1_prep.plant_name.str.match('.*mitchell.*')) &
                              (ferc1_prep.respondent_id==127)),
    
    ((ferc1_prep.plant_name.str.match('.*mitchell.*share$')) &
                              (ferc1_prep.respondent_id==81)),
    
    ((ferc1_prep.plant_name.str.match('.*mitchell.*share$')) &
                              (ferc1_prep.respondent_id==452)),
    
    ((ferc1_prep.plant_name.str.match('.*mitchell.*share$')) &
                              (ferc1_prep.respondent_id==192)),
    
    (ferc1_prep.plant_name.str.match('.*irginia city.*')),
    
    ((ferc1_prep.plant_name.str.match('.*east bend.*')) &
            (ferc1_prep.respondent_id==178)),
    
    ((ferc1_prep.plant_name.str.match('.*east bend.*')) &
            (ferc1_prep.respondent_id==27)),
    
    ((ferc1_prep.plant_name.str.match('.*east bend.*')) &
            (ferc1_prep.respondent_id==42)),
    
   ((ferc1_prep.plant_name.str.match('.*arrison.*')) &
            (ferc1_prep.respondent_id==101)),
    
    ((ferc1_prep.plant_name.str.match('^amos-opco share$')) &
            (ferc1_prep.respondent_id==127)),
    
    ((ferc1_prep.plant_name.str.match('^amos-apco share$')) &
            (ferc1_prep.respondent_id==6)),
    
    ((ferc1_prep.plant_name.str.match('^amos$')) &
            (ferc1_prep.respondent_id==6)),
    
    ((ferc1_prep.plant_name.str.match('.*killen.*')) &
            (ferc1_prep.respondent_id==27)),
    
    ((ferc1_prep.plant_name.str.match('.*killen.*')) &
            (ferc1_prep.respondent_id==42) &
            (ferc1_prep.tot_capacity > 100.0)),
  # This plant results in a doubling of the year columns. Not sure why but it is commented out.  
  # ((ferc1_prep.plant_name.str.match('.*stuart.*')) &
           # (ferc1_prep.respondent_id==31))         
        
    ((ferc1_prep.plant_name.str.match('.*stuart.*')) &
            (ferc1_prep.respondent_id==31)),
    
    ((ferc1_prep.plant_name.str.match('.*stuart.*')) &
            (ferc1_prep.respondent_id==127)),
    
    ((ferc1_prep.plant_name.str.match('.*stuart.*')) &
            (ferc1_prep.respondent_id==452)),
    
    ((ferc1_prep.plant_name.str.match('comanche')) &
            (ferc1_prep.respondent_id==145)),
    
    ((ferc1_prep.plant_name.str.match('pawnee')) &
            (ferc1_prep.respondent_id==145)),
    
    ((ferc1_prep.plant_name.str.match('hayden')) &
            (ferc1_prep.respondent_id==145)),
    
    ((ferc1_prep.plant_name.str.match('hayden')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('craig')) &
            (ferc1_prep.respondent_id==145)),
    
    ((ferc1_prep.plant_name.str.match('craig')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*')) &
            (ferc1_prep.respondent_id==122)),
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*')) &
            (ferc1_prep.respondent_id==187)),
    # Colstrip 1 & 2 and Colstrip 3 & 4 are owned by the same utility and
    # reported together. When only searching for '.*colstrip.*', the dataframe didn't have
    # the same shape to contatentate to the rest of the plants so instead the search is
    # for '.*colstrip.*2$' and '.*colstrip.*4$'.
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*2$')) &
            (ferc1_prep.respondent_id==150)),
    
    ((ferc1_prep.plant_name.str.match('.*colstrip.*4$')) &
            (ferc1_prep.respondent_id==150)),
    
    ((ferc1_prep.plant_name.str.match('^hunter.*1$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^hunter.*2$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^hunter.*3$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^jim bridger$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^jim bridger$')) &
            (ferc1_prep.respondent_id==70)),
    
    ((ferc1_prep.plant_name.str.match('^huntington$')) &
            (ferc1_prep.respondent_id==134)),
    
    ((ferc1_prep.plant_name.str.match('^meramec$')) &
            (ferc1_prep.respondent_id==177)),
    
    ((ferc1_prep.plant_name.str.match('^valmy$')) &
           (ferc1_prep.respondent_id==70)),
    
    ((ferc1_prep.plant_name.str.match('.*valmy.*')) &
           (ferc1_prep.respondent_id==157)),
    
    ((ferc1_prep.plant_name.str.match('^big stone$')) &
           (ferc1_prep.respondent_id==132)),
    
    ((ferc1_prep.plant_name.str.match('^big stone$')) &
           (ferc1_prep.respondent_id==122)),
    
    ((ferc1_prep.plant_name.str.match('^big stone$')) &
           (ferc1_prep.respondent_id==95)),
    
    ((ferc1_prep.plant_name.str.match('four corners')) &
            (ferc1_prep.respondent_id==49)),
    
    ((ferc1_prep.plant_name.str.match('four corners')) &
            (ferc1_prep.respondent_id==176)),
    
    ((ferc1_prep.plant_name.str.match('four corners')) &
            (ferc1_prep.respondent_id==161)),

    # From 2004 through 2006 respondent_id 7 splits the Four Corners plant into units
    # 1, 2, and 3, and units 4 and 5. The combined records for 1, 2, and 3 are used along
    # with units 1, 2, and 3, which appear on their own in later years. The combined records 
    # for 4 and 5 are used along with units 4 and 5, which also appear on their own in later years.
    
    ((ferc1_prep.plant_name.str.match('four corners 1')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7) &
            (ferc1_prep.yr_const=='1963')),
    
    ((ferc1_prep.plant_name.str.match('four corners 2')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7) &
            (ferc1_prep.yr_const=='1963')),
    
    ((ferc1_prep.plant_name.str.match('four corners 3')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7) &
            ((ferc1_prep.yr_const=='1963')|(ferc1_prep.yr_const=='1964'))),
    
    ((ferc1_prep.plant_name.str.match('four corners 4')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7)&
            (ferc1_prep.yr_const=='1969')),
    
    ((ferc1_prep.plant_name.str.match('four corners 5')|
             (ferc1_prep.plant_name=='four corners')) &
            (ferc1_prep.respondent_id==7)&
            (ferc1_prep.yr_const=='1969')),
    
    
    # Other plants:
    
    # gas plant, likely a CT
    
    ((ferc1_prep.plant_name.str.match('.*killen.*')) &
            (ferc1_prep.respondent_id==42) &
            (ferc1_prep.tot_capacity < 100.0)),
    
    # Bituminous coal is used as a primary fuel type, which can be substituted for natural gas
    # https://en.wikipedia.org/wiki/A._B._Brown_Generating_Station
    ((ferc1_prep.plant_name.str.match('.*brown.*')) &
            (ferc1_prep.respondent_id==163) &
            (ferc1_prep.yr_const=='1979')),
    
    # Gas turbine
    
    ((ferc1_prep.plant_name.str.match('.*brown.*')) &
            (ferc1_prep.respondent_id==163) &
            (ferc1_prep.yr_const=='1991')),
    
    # Gas turbine
    
    ((ferc1_prep.plant_name.str.match('.*brown.*')) &
            (ferc1_prep.respondent_id==163) &
            (ferc1_prep.yr_const=='2002')),
    
    #  Acadia Power Station - Cleco owns Acadia Unit 1 and half of the plant's common assets 
    # and operates and maintains the entire plant. Entergy Louisiana, LLC owns Unit 2 
    # and the other half of the plant's common assets.

    # Acadia began commercial operations in 2002. 
    # The 1,160-megawatt plant is fueled by natural gas and uses combined-cycle 
    # technology to produce efficient, reliable power.
    ((ferc1_prep.plant_name.str.match('.*acadia.*')) &
    (ferc1_prep.respondent_id==87)),
    
    ((ferc1_prep.plant_name.str.match('.*acadia.*')) &
    (ferc1_prep.respondent_id==454)),
    
    ((ferc1_prep.plant_name.str.match('.*acadia.*')) &
    (ferc1_prep.respondent_id==22)),
    
    # Barry coal
    
    ((ferc1_prep.plant_name.str.match('.*barry.*')) &
    (ferc1_prep.yr_const=='1954')),
    
    # Barry combined cycle
    
    ((ferc1_prep.plant_name.str.match('.*barry.*')) &
    (ferc1_prep.yr_const=='2000')),
    
    # Fort Churchill gas turbine
    # https://en.wikipedia.org/wiki/Fort_Churchill_Generating_Station
    
    ((ferc1_prep.plant_name.str.match('.*ft churchill.*')) &
    ((ferc1_prep.respondent_id==157))),
     
    # Tracy gas units Sierra Pacific
    # https://en.wikipedia.org/wiki/Frank_A._Tracy_Generating_Station
     
    ((ferc1_prep.plant_name.str.match('.*tracy.*1.*2.*3')) &
    ((ferc1_prep.respondent_id==157))),
     
    (ferc1_prep.plant_name.str.match('tracy 3')),
    
    # Tracy 8-10
    
    (ferc1_prep.plant_name.str.match('tracy 8-10')),
    
    # Tracy 4&5
     
    (ferc1_prep.plant_name.str.match('tracy 4&5')),
    
    # Wolf Creek nuclear plant Kansas Gas and Electric Company
    
    ((ferc1_prep.plant_name.str.match('^wolf creek total$')) &
   ((ferc1_prep.respondent_id==80))),
    
    ((ferc1_prep.plant_name.str.match('^wolf creek 47%$')) &
   ((ferc1_prep.respondent_id==80))),
    
    # Murray Gill gas turbine Kansas City Gas and Electric Company
    # https://www.westarenergy.com/natural-gas
    
    ((ferc1_prep.plant_name.str.match('^murray gill$')) &
   ((ferc1_prep.respondent_id==80))),
    
    # Cunningham gas steam turbine Southwestern Public Service Company
    # https://www.xcelenergy.com/energy_portfolio/electricity/power_plants/cunningham
    
    ((ferc1_prep.plant_name.str.match('cunningham steam')) &
   ((ferc1_prep.respondent_id==166))),
    
    # Cunningham gas combustion turbine Southwestern Public Service Company
    
    ((ferc1_prep.plant_name.str.match('cunningham gas turbs')) &
   ((ferc1_prep.respondent_id==166))),
    
    # Maddox gas steam turbine Southwestern Public Service Company
    # https://www.xcelenergy.com/energy_portfolio/electricity/power_plants/cunningham
    
    ((ferc1_prep.plant_name.str.match('maddox steam')) &
   ((ferc1_prep.respondent_id==166))),
    
    # Maddox gas combustion turbine Southwestern Public Service Company
    
    ((ferc1_prep.plant_name.str.match('maddox gas')) &
   ((ferc1_prep.respondent_id==166))),
    
    # Schiller coal Public Service Company of New Hampshire
    
    ((ferc1_prep.plant_name.str.match('schiller')) &
   ((ferc1_prep.respondent_id==146)) &
    (ferc1_prep.yr_const=='1947')),
    
    # Schiller gas turbine Public Service Company of New Hampshire
    
    ((ferc1_prep.plant_name.str.match('schiller')) &
   ((ferc1_prep.respondent_id==146)) &
    (ferc1_prep.yr_const=='1970')),
    
   # Mustang Station combined cycle Golden Spread Electric Cooperative, Inc.
    
    ((ferc1_prep.plant_name.str.match('^mustang station$')) &
   ((ferc1_prep.respondent_id==58)) &
    (ferc1_prep.plant_kind=='Combined Cycle')),
    
    # Mustang Station gas turbine Golden Spread Electric Cooperative, Inc.
    
    ((ferc1_prep.plant_name.str.match('^mustang station$')) &
   ((ferc1_prep.respondent_id==58)) &
    (ferc1_prep.plant_kind=='Gas Turbine')),
    
    # Brunswick nuclear Duke Energy Progress, Inc.
    
    ((ferc1_prep.plant_name.str.match('brunswick')) &
   ((ferc1_prep.respondent_id==17))),
    
    # O.H. Hutchings coal Dayton Power and Light Company
    
    ((ferc1_prep.plant_name.str.match('^o.h. hutchings$')) &
   ((ferc1_prep.respondent_id==42))),
    
    # Saint Clair natural gas peaker DTE Electric Company
    
    ((ferc1_prep.plant_name.str.match('.*clair.*')) &
   ((ferc1_prep.respondent_id==44)) &
    (ferc1_prep.tot_capacity==18.59)),
    
    # Saint Clair coal DTE Electric Company
    
    ((ferc1_prep.plant_name.str.match('st. clair pp')) &
   ((ferc1_prep.respondent_id==44)))
    
]

In [444]:
ferc1_train = pd.DataFrame({}, columns=range(2004,2017))
new_plants = [ferc1_train]
for mask in masks:
    new_plants = new_plants + [ferc1_prep[mask][['record_id','report_year']].\
                                 set_index('report_year').transpose()]
ferc1_train = pd.concat(new_plants).reset_index().drop('index', axis=1)
ferc1_train

report_year,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,2004_144_0_5,2005_144_0_5,2006_144_0_5,2007_144_0_5,2008_144_0_5,2009_144_0_5,2010_144_0_5,2011_144_1_4,2012_144_1_4,2013_144_1_4,2014_144_1_4,2015_144_1_4,2016_144_1_4
1,2004_148_0_5,2005_148_0_5,2006_148_0_5,2007_148_0_5,2008_148_0_5,2009_148_0_5,2010_148_0_5,2011_148_0_5,2012_148_0_5,2013_148_0_5,2014_148_0_5,2015_148_0_5,2016_148_0_5
2,2004_22_0_3,2005_22_0_3,2006_22_0_3,2007_22_0_3,2008_22_0_3,2009_22_0_3,2010_22_0_3,2011_22_0_3,2012_22_0_3,2013_22_0_3,2014_22_0_3,2015_22_0_3,2016_22_0_3
3,2004_148_1_2,2005_148_1_2,2006_148_1_2,2007_148_1_2,2008_148_1_2,2009_148_1_2,2010_148_1_2,2011_148_1_2,2012_148_1_2,2013_148_1_2,2014_148_1_2,2015_148_1_2,2016_148_1_2
4,2004_189_0_4,2005_189_0_4,2006_189_0_1,2007_189_0_1,2008_189_0_1,2009_189_0_1,2010_189_0_1,2011_189_0_1,2012_189_0_1,2013_189_0_1,2014_189_0_1,2015_189_0_1,2016_189_0_1
5,2004_24_2_1,2005_24_2_1,2006_24_0_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2004_191_1_4,2005_191_1_2,2006_191_1_4,2007_191_1_4,2008_191_1_4,2009_191_1_4,2010_191_1_4,2011_191_1_4,2012_191_1_4,2013_191_1_3,2014_191_1_3,2015_191_1_3,2016_191_1_3
7,2004_80_1_2,2005_80_1_2,2006_80_1_2,2007_80_1_2,2008_80_1_2,2009_80_1_2,2010_80_1_1,2011_80_1_1,2012_80_1_1,2013_80_0_5,2014_80_0_5,2015_80_0_5,2016_80_0_5
8,2004_182_0_3,2005_182_0_3,2006_182_0_3,2007_182_0_3,2008_182_0_3,2009_182_0_3,2010_182_0_3,2011_182_0_3,2012_182_0_3,2013_182_0_3,2014_182_0_3,2015_182_0_3,2016_182_0_3
9,2004_79_0_4,2005_79_0_4,2006_79_0_4,2007_79_0_4,2008_79_0_4,2009_79_2_1,2010_79_2_3,2011_79_2_3,2012_79_2_3,2013_79_2_3,2014_79_2_3,2015_79_2_3,2016_79_2_3


In [442]:
new_mask = (
     ((ferc1_prep.plant_name.str.match('st. clair pp')) &
   ((ferc1_prep.respondent_id==44))))
ferc1_prep[new_mask]

,record_id,report_year,spplmnt_num,row_number,respondent_id,plant_name,plant_kind_cpi,yr_const,tot_capacity,index,plant_kind,type_const,yr_installed,peak_demand,plant_hours,plnt_capability,when_not_limited,when_limited,avg_num_of_emp,net_generation,cost_land,cost_structure,cost_equipment,cost_of_plant_to,cost_per_kw,expns_operations,expns_fuel,expns_coolants,expns_steam,expns_steam_othr,expns_transfer,expns_electric,expns_misc_power,expns_rents,expns_allowances,expns_engnr,expns_structures,expns_boiler,expns_plants,expns_misc_steam,tot_prdctn_expns,expns_kwh,asset_retire_cost
143,2004_44_2_4,2004,2,4,44,st. clair pp,steam,1953,1905.01,245,Steam,Conventional,1969,1296.0,8784.0,0.0,1415.0,1415.0,405.0,7.388219e+09,1721682.0,50275191.0,687034066.0,7.391218e+08,387.9884,1172933.0,114438731.0,0.0,12731686.0,0.0,0.0,706681.0,6079319.0,0.0,0.0,4108781.0,3271189.0,23446711.0,7469718.0,5010837.0,178436586.0,0.0242,90905.0
1685,2005_44_2_4,2005,2,4,44,st. clair pp,steam,1953,1905.01,3168,Steam,Conventional,1969,1296.0,8760.0,0.0,1415.0,1415.0,401.0,7.378283e+09,1721682.0,50955507.0,668428853.0,7.225794e+08,379.3048,831927.0,111447253.0,0.0,6812494.0,0.0,0.0,316610.0,7337501.0,0.0,0.0,2100548.0,6525848.0,16907985.0,1746364.0,4884797.0,158911327.0,0.0215,1473406.0
2400,2006_44_2_4,2006,2,4,44,st. clair pp,steam,1953,1905.01,4443,Steam,Conventional,1969,1417.0,8760.0,0.0,1417.0,1366.0,373.0,7.439193e+09,1721682.0,46389871.0,696697001.0,7.463975e+08,391.8077,828636.0,107581723.0,0.0,1924451.0,0.0,0.0,42377.0,13640173.0,0.0,0.0,430937.0,3442915.0,27372978.0,8368725.0,5921637.0,169554552.0,0.0228,1588946.0
3884,2007_44_2_4,2007,2,4,44,st. clair pp,steam,1953,1905.01,7203,Steam,Conventional,1969,1417.0,8760.0,0.0,1417.0,1366.0,373.0,7.614770e+09,2637153.0,51492068.0,734194789.0,7.899402e+08,414.6646,6495423.0,109108330.0,0.0,2543453.0,0.0,0.0,1511328.0,10151766.0,0.0,0.0,326753.0,4372928.0,23595002.0,2016912.0,6084185.0,166206080.0,0.0218,1616234.0
4108,2008_44_2_4,2008,2,4,44,st. clair pp,steam,1953,1905.01,7623,Steam,Conventional,1969,1417.0,8784.0,1417.0,1417.0,1368.0,327.0,7.676829e+09,2531472.0,14265726.0,783690950.0,8.006873e+08,420.3061,3015318.0,133406633.0,0.0,3033368.0,0.0,0.0,2021911.0,8602215.0,0.0,0.0,167804.0,1892983.0,21939810.0,3250923.0,4387996.0,181718961.0,0.0237,199194.0
5343,2009_44_2_4,2009,2,4,44,st. clair pp,steam,1953,1905.01,10033,Steam,Conventional,1969,1414.0,8760.0,1414.0,1414.0,1365.0,323.0,6.576120e+09,2637153.0,56331404.0,765672942.0,8.279842e+08,434.6351,2508915.0,110691252.0,0.0,2910294.0,0.0,0.0,1938885.0,9050850.0,0.0,0.0,166822.0,2039836.0,28884397.0,7019678.0,4391816.0,169602745.0,0.0258,3342717.0
6498,2010_44_2_4,2010,2,4,44,st. clair pp,steam,1953,1905.01,11998,Steam,Conventional,1969,1414.0,8760.0,1414.0,1414.0,1374.0,312.0,6.788349e+09,2637153.0,57264162.0,778472416.0,8.559377e+08,449.3088,2099656.0,130804910.0,0.0,2867606.0,0.0,0.0,1938976.0,9337638.0,0.0,0.0,296989.0,1934297.0,21074980.0,2773469.0,3498442.0,176626963.0,0.0260,17563942.0
7424,2011_44_2_4,2011,2,4,44,st. clair pp,steam,1953,1905.01,13733,Steam,Conventional,1969,1414.0,8760.0,1414.0,1414.0,1386.0,12.0,6.137136e+09,2637153.0,53729905.0,760854946.0,8.344920e+08,438.0512,2364107.0,152214337.0,0.0,3077816.0,0.0,0.0,2067671.0,10484028.0,0.0,0.0,156888.0,3767806.0,17970814.0,4094348.0,5386955.0,201584770.0,0.0328,17269971.0
8510,2012_44_2_4,2012,2,4,44,st. clair pp,steam,1953,1905.01,15773,Steam,Conventional,1969,1414.0,8784.0,1414.0,1414.0,1414.0,299.0,5.428600e+09,1717828.0,55310620.0,777466520.0,8.656944e+08,454.4304,2240349.0,156947484.0,0.0,3263564.0,0.0,0.0,2198626.0,10167276.0,0.0,0.0,156586.0,2242709.0,19510773.0,4792240.0,4513766.0,206033373.0,0.0380,31199413.0
9465,2013_44_2_4,2013,2,4,44,st. clair pp,steam,1953,1905.01,17693,Steam,Conventional,1969,1414.0,8760.0,1414.0,1414.0,1379.0,345.0,6.242818e+09,1717828.0,56354242.0,789053725.0,8.811823e+08,462.5605,2221694.0,177270526.0,0.0,3594369.0,0.0,0.0,2435695.0,11046279.0,0.0,15712237

In [6]:
# In this cell, you can mess around with various selections from the ferc plant
# dataframe, until it's just getting the plants you want to keep, and then add
# your new mask to the list of masks above.
#new_mask = (
#    (ferc1_prep.respondent_id==210) &
#    (ferc1_prep.yr_const=='2003')
#)
#ferc1_prep[new_mask]

In [9]:
ferc1_train.to_csv('ferc1_plantid_training.csv', index=False)

This is the suggested mask code:

## FERC Masks from CES work
These masks won't work directly because of different dataframe names and available columns and column names... but they should give you hints on how to go grab these same plants.

In [ ]:
ferc_masks['rodemacher'] = (ferc_df.plant_name.str.match('.*odemach.*2'))

ferc_masks['oklaunion_psok'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                (ferc_df.utility_id==148))
ferc_masks['oklaunion_aeptx_north'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                       (ferc_df.utility_id==189))
ferc_masks['oklaunion_aeptx_central'] = ((ferc_df.plant_name.str.match('.*klaunion.*')) &
                                         (ferc_df.utility_id==24))

ferc_masks['jeffrey_westar'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                                (ferc_df.utility_id==191) &
                                (ferc_df.expns_fuel > 0.0))
ferc_masks['jeffrey_ksgeco'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                                (ferc_df.utility_id==80) &
                                (ferc_df.expns_fuel > 0.0))
ferc_masks['jeffrey_kcpl'] = ((ferc_df.plant_name.str.match('.*effrey.*')) &
                              (ferc_df.utility_id==182) &
                              (ferc_df.expns_fuel > 0.0))

ferc_masks['lacygne_1_kcpl'] = (
    (ferc_df.plant_name.str.match('(La Cygne 50%|Lacygne 50%|Lacygne 1.*)')) &
    (ferc_df.utility_id==79)
)
ferc_masks['lacygne_2_kcpl'] = ((ferc_df.plant_name.str.match('.*ygne.*2.*')) &
                               (ferc_df.utility_id==79))
ferc_masks['lacygne_1_kge'] = ((ferc_df.plant_name.str.match('.*ygne.*#1.*')) &
                              (ferc_df.utility_id==80))
ferc_masks['lacygne_2_kge'] = ((ferc_df.plant_name.str.match('.*ygne.*#2.*')) &
                              (ferc_df.utility_id==80))

ferc_masks['sibley'] = ((ferc_df.utility_id==182) &
                        (ferc_df.plant_name=='Sibley'))

ferc_masks['george_neal_1'] = ((ferc_df.plant_name.str.match('.*Neal.*1')))
ferc_masks['george_neal_2'] = ((ferc_df.plant_name.str.match('.*Neal.*2')))
ferc_masks['george_neal_3_ipl'] = (
    (ferc_df.plant_name.str.match('.*Neal.*3') &
    (ferc_df.utility_id==281)))
ferc_masks['george_neal_3_midam'] = (
    (ferc_df.plant_name.str.match('.*Neal.*3') &
    (ferc_df.utility_id==210)))

ferc_masks['san_juan_psnm'] = ((ferc_df.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
                               (ferc_df.utility_id==147))
ferc_masks['san_juan_tepco'] = ((ferc_df.plant_name.str.match('.*[Ss]an [Jj]uan.*')) &
                                (ferc_df.utility_id==176))

ferc_masks['yorktown'] = (ferc_df.plant_name.str.match('.*orktown.*'))

ferc_masks['chesterfield'] = (ferc_df.plant_name.str.match('Chesterfield$'))

ferc_masks['rockport_aeg'] = ((ferc_df.plant_name.str.match('Rockport Total Aeg')) &
                              (ferc_df.utility_id==1))
ferc_masks['rockport_inmi'] = ((ferc_df.plant_name.str.match('Rockport Total I&M')) &
                               (ferc_df.utility_id==73))

ferc_masks['mt_storm'] = ((ferc_df.plant_name.str.match('Mount Storm$')) &
                          (ferc_df.utility_id==186))

###########################################################################
# Mitchell (West Virginia) plant. Originally owned by Ohio Power Company,
# then split between Kentucky Power Company, and AEP, subsequently Wheeling
ferc_masks['mitchell_opc'] = ((ferc_df.plant_name.str.match('.*[Mm]itchell.*')) &
                              (ferc_df.utility_id==127))
ferc_masks['mitchell_kpc'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==81))
ferc_masks['mitchell_aep'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==452))
ferc_masks['mitchell_wpc'] = ((ferc_df.plant_name.str.match('.*Mitchell.*Share$')) &
                              (ferc_df.utility_id==192))

ferc_masks['virginia_city'] = ((ferc_df.plant_name.str.match('.*irginia [Cc]ity.*')))

###########################################################################
# East Bend -- Ownership of Duke's share seems to have moved from one
# subsidiary to another. Also, why is only generator_id=2 showing up in EIA?
# Seems like another case of some bad NA values.
ferc_masks['east_bend_dukeoh'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                  (ferc_df.utility_id==27))
ferc_masks['east_bend_daypl'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                 (ferc_df.utility_id==42))
ferc_masks['east_bend_dukeky'] = ((ferc_df.plant_name.str.match('.*East Bend.*')) &
                                  (ferc_df.utility_id==178))

###########################################################################
# Harrison plant, owned by FirstEnergy, sold to subsidiary Mon Energy in
# 2013 -- at that point FERC reported capacity jumps to 2000MW. Only a 
# couple of complete years of cost data in EIA -- suffers from NA values.
# Last 2 years of FERC data are totally wacked -- neeeds to be cleaned up.
# Also, capacity reported doesn't really match up between EIA & FERC... we're
# missing a third unit, for which there's *no* whole year of data that's valid
# in EIA.  Gah.
ferc_masks['harrison'] = ((ferc_df.plant_name.str.match('.*arrison.*')) &
                                  (ferc_df.utility_id==101))

###########################################################################
# John Amos plant has 3 units. Jointly owned by Appalachian Power Co & 
# Ohio Power Co until 2013, after which point it is entirely App Pwr Co.
ferc_masks['amos_apco'] = ((ferc_df.plant_name.str.match('^Amos-Apco Share$')) &
                           (ferc_df.utility_id==6))
ferc_masks['amos_opco'] = ((ferc_df.plant_name.str.match('^Amos-Opco Share$')) &
                           (ferc_df.utility_id==127))
ferc_masks['amos_consolidated'] = ((ferc_df.plant_name.str.match('^Amos$')) &
                                   (ferc_df.utility_id==6))

###########################################################################
# Killen is slated for retirement mid-2018.  There's a weird dangling
# 12-18 MW generator of some kind being reported to FERC, which is why the
# DPL filter includes the nameplate capacity minimum of 100.0 MW. DPL also
# has fuel costs which are off by a factor of 1000x in 2015 only.
ferc_masks['killen_duke'] = ((ferc_df.plant_name.str.match('.*Killen.*')) &
                             (ferc_df.utility_id==27))
ferc_masks['killen_dpl'] = ((ferc_df.plant_name.str.match('.*Killen.*')) &
                              (ferc_df.utility_id==42) &
                              (ferc_df.nameplate_capacity_mw > 100.0))

###########################################################################
# J.M. Stuart plant has 4 coal fired units, and an ownership structure that
# has changed over the years. 2015 fuel cost numbers reported by DPL are off
# by a factor of 1000x.
ferc_masks['jm_stuart_duke'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                                (ferc_df.utility_id==27))
ferc_masks['jm_stuart_csp'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==31))
ferc_masks['jm_stuart_opco'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                                (ferc_df.utility_id==127))
ferc_masks['jm_stuart_aep'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==452))
ferc_masks['jm_stuart_dpl'] = ((ferc_df.plant_name.str.match('.*Stuart.*')) &
                               (ferc_df.utility_id==42) &
                               (ferc_df.nameplate_capacity_mw > 100.0))

ferc_masks['comanche'] = ((ferc_df.plant_name.str.match('Comanche')) &
                          (ferc_df.utility_id==145))

ferc_masks['pawnee'] = ((ferc_df.plant_name.str.match('Pawnee')) &
                        (ferc_df.utility_id==145))

###########################################################################
# Hayden 1 & 2 -- operated by PSCO, partly owned by PacifiCorp
ferc_masks['hayden_psco'] = ((ferc_df.plant_name.str.match('Hayden')) &
                             (ferc_df.utility_id==145))
ferc_masks['hayden_pc'] = ((ferc_df.plant_name.str.match('Hayden')) &
                           (ferc_df.utility_id==134))

###########################################################################
# Craig station is a big ownership mess...
ferc_masks['craig_psco'] = ((ferc_df.plant_name.str.match('Craig')) &
                            (ferc_df.utility_id==145))
ferc_masks['craig_pc'] = ((ferc_df.plant_name.str.match('Craig')) &
                          (ferc_df.utility_id==134))

###########################################################################
# Colstrip is complicated. No EIA data for some reason -- cost are all
# unreported, but there should be other data, no? But it's all NA. I have
# included what I think the masks should be to capture those units anyway,
# in case we can fix it later or there's some real data there monthly...
# FERC ownership adds up to 1400MW in 2008 and later, but less earlier.
# Did the plant used to be largely owned by a public power entity?
ferc_masks['colstrip_nwc'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                              (ferc_df.utility_id==122)) # 240 MW 2004-2016
ferc_masks['colstrip_pc'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                             (ferc_df.utility_id==134)) # 155 MW 2004-2016
ferc_masks['colstrip_avista'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                                 (ferc_df.utility_id==187)) # 233 MW 2004-2016
ferc_masks['colstrip_pse'] = ((ferc_df.plant_name.str.match('.*Colstrip.*')) &
                              (ferc_df.utility_id==150)) # 377 + 433 MW 2008-2016

###########################################################################
# Hunter plant. No EIA data coming through on 2014-2015 at annual freq.
# PacifiCorp is reporting the three units independently for some reason,
# which is great.
ferc_masks['hunter_1'] = ((ferc_df.plant_name.str.match('^Hunter.*1$')) &
                          (ferc_df.utility_id==134))
ferc_masks['hunter_2'] = ((ferc_df.plant_name.str.match('^Hunter.*2$')) &
                          (ferc_df.utility_id==134))
ferc_masks['hunter_3'] = ((ferc_df.plant_name.str.match('^Hunter.*3$')) &
                          (ferc_df.utility_id==134))

###########################################################################
# Jim Bridger -- 4 units, jointly owned by PacifiCorp & Idaho Power Co.
ferc_masks['jim_bridger_pc'] = ((ferc_df.plant_name.str.match('^Jim Bridger$')) &
                                (ferc_df.utility_id==134))
ferc_masks['jim_bridger_idpc'] = ((ferc_df.plant_name.str.match('^Jim Bridger$')) &
                                  (ferc_df.utility_id==70))

ferc_masks['huntington'] = ((ferc_df.plant_name.str.match('^Huntington$')) &
                            (ferc_df.utility_id==134))

###########################################################################
# Maramec - 4 units, one owner
ferc_masks['maramec'] = ((ferc_df.plant_name.str.match('^Meramec$')) &
                            (ferc_df.utility_id==177))

###########################################################################
# North Valmy - 2 units, jointly owned by Sierra Pacific & Idaho Power
ferc_masks['north_valmy_sp'] = ((ferc_df.plant_name.str.match('^Valmy$')) &
                            (ferc_df.utility_id==157))
ferc_masks['north_valmy_ip'] = ((ferc_df.plant_name.str.match('^Valmy$')) &
                            (ferc_df.utility_id==70))


###########################################################################
# Big Stone, three FERC respondents
ferc_masks['big_stone_ot'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==132))
ferc_masks['big_stone_nwc'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==122))
ferc_masks['big_stone_mdu'] = ((ferc_df.plant_id_pudl==55) &
                                (ferc_df.utility_id==95))

###########################################################################
# Four Corners... 5 units. 5 owners. APS reports units after 2006
ferc_masks['four_courners_elp'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==49))
ferc_masks['four_courners_tep'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==176))
ferc_masks['four_courners_sce'] = (ferc_df.plant_name.str.match('Four Corners')&
                                     (ferc_df.utility_id==161))

ferc_masks['four_courners_1_aps'] = (ferc_df.plant_name.str.match('Four Corners 1')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_2_aps'] = (ferc_df.plant_name.str.match('Four Corners 2')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_3_aps'] = (ferc_df.plant_name.str.match('Four Corners 3')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_4_aps'] = (ferc_df.plant_name.str.match('Four Corners 4')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))
ferc_masks['four_courners_5_aps'] = (ferc_df.plant_name.str.match('Four Corners 5')
                                     |(ferc_df.plant_name=='Four Corners') &
                                     (ferc_df.utility_id==7))

ferc_masks['four_courners_4_4c'] = (ferc_df.plant_name.str.match('Four Corners4')&
                                     (ferc_df.utility_id==147))
ferc_masks['four_courners_5_4c'] = (ferc_df.plant_name.str.match('Four Corners 5')&
                                     (ferc_df.utility_id==147))


ferc_masks['four_courners_1_psnm'] = (ferc_df.plant_name.str.match('Four Corners (1)')&
                                     (ferc_df.utility_id==513))

